# (심화과제) Multi-genre natural language inference(MNLI)


In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

In [4]:
import torch
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main


In [228]:
import logging as log

for handler in log.root.handlers[:]:
    log.root.removeHandler(handler)

log.basicConfig(
    level= log.INFO,
)

device = 'mps'
log.debug(device)

[MYCODE] nyu-mll/multi_nli 데이터셋 불러오기 

- 데이터셋의 구조 파악하기 

In [ ]:
ds = load_dataset("nyu-mll/multi_nli")

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/nyu-mll/multi_nli HTTP/1.1" 200 2731
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /datasets.huggingface.co/datasets/datasets/nyu-mll/multi_nli/nyu-mll/multi_nli.py HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/datasets/nyu-mll/multi_nli HTTP/1.1" 200 2731
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/nyu-mll/multi_nli/resolve/da70db2af9d09693783c3320c4249840212ee221/README.md HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /datasets/nyu-mll/multi_nli/resolve/da70db2af9d09693783c3320c4249840212ee221/.huggingface.yaml HTTP/1.1" 404 0
DEBUG:urllib3.connectionp

[MYCODE] 데이터셋 구조 확인
- ds 는 train, validation_matched, validation_mismatched 의 데이터를 가지고 있습니다.
- 레이블 분표율을 확인해본 결과 균등하게 있음을 확인 

[FEEDBACK]
- 만약 레이블의 데이터가 균등하지 않다면 어떻게 해야할까요?
- 균등하지 않는 데이터를 하기 위해서 어떤 작업을 하면 좋나요 ?

In [171]:
log.debug(ds)

# MemoryMappedTable 
log.debug(f"type = {type(ds.data['train'])}")

# 학습 데이터의 레이블 분포 확인
labels = [example['label'] for example in ds['train']]
log.debug("Label Distribution:", Counter(labels))

# 데이터셋 구조를 확인합니다.
log.debug(f"{help(ds['train'])}")

DEBUG:root:DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
        num_rows: 9832
    })
})
DEBUG:root:type = <class 'datasets.table.MemoryMappedTable'>
DEBUG:root:None


Label Distribution: Counter({2: 130903, 1: 130900, 0: 130899})
Help on Dataset in module datasets.arrow_dataset object:

class Dataset(DatasetInfoMixin, datasets.search.IndexableMixin, TensorflowDatasetMixin)
 |  Dataset(arrow_table: datasets.table.Table, info: Optional[datasets.info.DatasetInfo] = None, split: Optional[datasets.splits.NamedSplit] = None, indices_table: Optional[datasets.table.Table] = None, fingerprint: Optional[str] = None)
 |  
 |  A Dataset backed by an Arrow table.
 |  
 |  Method resolution order:
 |      Dataset
 |      DatasetInfoMixin
 |      datasets.search.IndexableMixin
 |      TensorflowDatasetMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, exc_type, exc_val, exc_tb)
 |  
 |  __getitem__(self, key)
 |      Can be used to index columns (by string names) or rows (by integer index or iterable of indices or bools).
 |  
 |  __getitems__(self, keys: List) -> List
 |      Can be

In [8]:
ds['train'].features

{'promptID': Value(dtype='int32', id=None),
 'pairID': Value(dtype='string', id=None),
 'premise': Value(dtype='string', id=None),
 'premise_binary_parse': Value(dtype='string', id=None),
 'premise_parse': Value(dtype='string', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'hypothesis_binary_parse': Value(dtype='string', id=None),
 'hypothesis_parse': Value(dtype='string', id=None),
 'genre': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)}

In [13]:
def to_tokenizer_fn(text, max_length=128):
    return tokenizer(text, padding=True, truncation=True, max_length=max_length)

def to_long_tensor_fn(input_tokenizer):
    return torch.LongTensor(input_tokenizer)

In [56]:
def collate_fn(batch):
  
  texts, labels = [], []
  for row in batch:
    texts.append(f"{row['premise']} [SEP] {row['hypothesis']}")
    labels.append(row['label'])
  
  token = to_long_tensor_fn(to_tokenizer_fn(texts, 128).input_ids)
  attention_mask = to_long_tensor_fn(to_tokenizer_fn(texts).attention_mask)
  label = to_long_tensor_fn(labels)
  log.debug(token)
  log.debug(attention_mask)
  log.debug(label)
  return token, attention_mask, label


[MYCDOE] 데이터로더를 생성

- 학습데이터
- 검증 데이터
- 빠르게 학습하기 위한 데이터로더

In [200]:
def create_balanced_subset(dataset, samples_per_label):
    """
    각 레이블별로 균등하게 샘플링된 서브셋을 생성.
    
    Args:
        dataset: 전체 데이터셋 (HuggingFace Dataset 형태)
        samples_per_label: 각 레이블당 샘플링할 개수
    
    Returns:
        균등하게 샘플링된 데이터셋 (Dataset 객체)
    """
    from collections import defaultdict
    import random

    # 레이블별 데이터 분류
    label_to_samples = defaultdict(list)
    for example in dataset:
        label_to_samples[example['label']].append(example)

    # 각 레이블에서 필요한 개수만큼 샘플링
    balanced_samples = []
    for label, samples in label_to_samples.items():
        if len(samples) < samples_per_label:
            raise ValueError(f"Not enough samples for label {label}.")
        balanced_samples.extend(random.sample(samples, samples_per_label))

    return Dataset.from_list(balanced_samples)

In [216]:
# 데이터 학습 데이터셋
train_loader = DataLoader(
    ds['train'], batch_size=128, shuffle=True, collate_fn=collate_fn
)

# 데이터 검증 데이터셋 
test_loader = DataLoader(
    ds['validation_matched'], batch_size=128, shuffle=True, collate_fn=collate_fn
)

# 빠른 모델 검증을 위한 작은 수의 데이터만을 학습
train_loader_small = create_balanced_subset(ds['train'], 400)
train_loader_small = DataLoader(
    train_loader_small, batch_size=128, shuffle=False, collate_fn=collate_fn
)

# 빠른 모델 검증을 위한 작은 수의 데이터만을 학습
test_loader_small = create_balanced_subset(ds['validation_matched'], 40)
test_loader_small = DataLoader(
    test_loader_small, batch_size=128, shuffle=False, collate_fn=collate_fn
)

log.debug(train_loader.dataset)
log.debug(test_loader.dataset)
log.debug(train_loader_small.dataset)
log.debug(test_loader_small.dataset)

# 학습 데이터의 레이블 분포 확인

DEBUG:root:Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 392702
})
DEBUG:root:Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 9815
})
DEBUG:root:Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 1200
})
DEBUG:root:Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 120
})


[MYCODE] 학습할 데이터의 정보를 확인합니다. 

- 두개의 문장을 하나로 합치는 작업을 선행 확인해보기 
    - 두 문장을 구분하는 값이 SEP 102으로 분리 되었는지 확인해보았습니다.


[MYCODE] train_dataloaders 에 따라 loader 변경 기능 추가 
- train_dataLoaders 할당하는 loader로 학습 변경하기 위해 선언 

In [222]:
train_dataloaders = train_loader_small
test_dataloaders = test_loader_small
log.debug(f"test_loader = {train_dataloaders.dataset}")

[MYCODE] callate_fn 변경으로 인한 데이터 검증 확인을 위해 데이터 확인 

[FEEDBACK]
- 데이터를 검증하는 작업을 눈으로 확인하게 되었는데 python 또는 AI 모델에서 데이터를 검증하기 위한 수단이 무엇이 있을까요 
    - TDD 같은 개념의 방법이 있나요?
    - assert 검증을 하는 방식인지 궁금합니다.

In [223]:
dataset_premise = train_dataloaders.dataset['premise']
dataset_hypothesis = train_dataloaders.dataset["hypothesis"]
dataset_labels = train_dataloaders.dataset["label"]

# 데이터 구조확인
log.debug(f"dataset_premise =  {dataset_premise}")
log.debug(f"dataset_hypothesis = {dataset_hypothesis}")
log.debug(f"dataset_labels = {dataset_labels}")

# 토크나이저 102 값이 중간에 넣어졌는지 확인하기
log.debug(f"{to_tokenizer_fn(dataset_premise)}")
log.debug(f"{to_tokenizer_fn(dataset_hypothesis)}")
log.debug(f"{collate_fn(train_dataloaders.dataset)}")

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [175]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [225]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, 3)

  def forward(self, x, attention_mask): 
    x = self.encoder(input_ids=x, attention_mask=attention_mask)
    return self.classifier(x.last_hidden_state[:, 0])

In [226]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [19]:
import numpy as np
import matplotlib.pyplot as plt
def live_plot(train_accs, test_accs, title="Real-time Accuracy", label1='train', label2='test'):
    # 플롯 크기 설정
    plt.figure(figsize=(10, 8))

    # 1번째 서브플롯: Train_loss vs test
    plt.subplot(2, 1, 1)
    x = np.arange(len(train_accs))
    plt.plot(x, train_accs, label='train', color='blue')  # 훈련 손실
    plt.plot(x, test_accs, label='test', color='red', marker='o')  # 검증 손실
    plt.title("Train_acc vs test_acc")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()


    # 플롯 간격 조정 및 출력
    plt.tight_layout()
    plt.show()

In [20]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)

    preds = model(inputs, attention_mask)
    preds = torch.argmax(preds, dim=-1)
    #preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [21]:
def save_checkpoint(epoch, model, optimizer, best_loss, counter, train_losses, train_accs, test_accs, filepath):
    """
    학습 상태를 저장하는 함수
    """
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_loss': best_loss,
        'counter': counter,
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_accs': test_accs
    }
    torch.save(checkpoint, filepath)
    log.info(f"Checkpoint saved at {filepath}")

In [22]:
def load_checkpoint(filepath, model, optimizer):
    """
    저장된 학습 상태를 복구하는 함수
    """
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    best_loss = checkpoint['best_loss']
    counter = checkpoint['counter']
    train_losses = checkpoint['train_losses']
    train_accs = checkpoint['train_accs']
    test_accs = checkpoint['test_accs']
    log.info(f"Checkpoint loaded from {filepath}")
    return epoch, best_loss, counter, train_losses, train_accs, test_accs

[MYCODE] 학습 정보가 

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

model = TextClassifier()

lr = 0.001
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)
n_epochs = 10

best_loss = float('inf')
patience = 3
train_losses = []
train_accs = []
test_accs = []

for epoch in range(n_epochs):
    total_loss = 0.0
    total_samples = 0
    model.train()
    for data in tqdm(train_dataloaders):
        model.zero_grad()
        inputs, attention_mask, labels = data
        inputs, attention_mask, labels = inputs.to(device), attention_mask.to(device), labels.to(device)

        preds = model(inputs, attention_mask)
        loss = loss_fn(preds, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        total_samples += inputs.size(0)

    avg_loss = total_loss / total_samples
    train_losses.append(avg_loss)

    # 손실 감소 확인
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        log.info(f"Epoch {epoch + 1}: avg improved to {avg_loss:.3f}.")
    else:
        counter += 1
        log.info(f"Epoch {epoch + 1}: Loss did not improve for {counter} consecutive epochs.")

    # 학습 중단 조건
    if counter >= patience:
        log.info(f"Stopping early at epoch {epoch + 1}. Best Loss: {best_loss:.3f}")
        break

    # 학습 상태 저장
    if epoch % 5 == 0:
        save_path = f"checkpoint_epoch_{epoch + 1}.pth"
        save_checkpoint(epoch + 1, model, optimizer, best_loss, counter, train_losses, train_accs, test_accs, save_path)

    # 모델 평가
    model.eval()
    train_acc = accuracy(model, train_dataloaders)
    test_acc = accuracy(model, test_dataloaders)
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    # 시각화와 로그
    if epoch % 5 == 0:
        live_plot(train_accs, test_accs)
    log.info(f"Epoch {epoch + 1} | Train Loss: {avg_loss:.3f} | Train Acc: {train_acc:.3f} | Test Acc: {test_acc:.3f}")

Using cache found in /Users/naseunghoo/.cache/torch/hub/huggingface_pytorch-transformers_main
100%|██████████| 10/10 [05:15<00:00, 31.60s/it]
INFO:root:Epoch 1: avg improved to 1.826.
INFO:root:Checkpoint saved at checkpoint_epoch_1.pth
